In [1]:
import tensorflow as tf
import datetime

import numpy as np
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [40]:
import tensorboard
%load_ext tensorboard 
%rm -rf ./logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
np.random.seed(42)
X = np.random.rand(100, 10)  # 100 10-dimensional observations
# binary label classification for each observation
y = np.random.randint(0, 2, size=100)

In [42]:
unit_range = np.arange(100, 105, 10)


In [43]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        loss = logs["loss"]
        if loss <= self.threshold:
            self.model.stop_training = True

In [44]:
initial_weights = []
final_weights = []
diff_weights = []

In [45]:
for units in unit_range:
    print(f"Number of units: {units}")
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=units, activation='relu', input_shape=(10,)),
        tf.keras.layers.Dense(units = units, activation = 'relu'),
        tf.keras.layers.Dense(units=1, activation='sigmoid')])

    # Print initial weights:
    initial_weights.append(model.layers[0].get_weights()[0])
    # Compile model with loss function and optimizer:

    # Instantiate the callback:
    my_callback = MyThresholdCallback(threshold=0.005)
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(X, y, epochs=400, batch_size=1, callbacks=my_callback , verbose=1)

    # Print out final weights and compute differences:
    final_weights.append(model.layers[0].get_weights()[0])
    diff_weights.append(final_weights[-1] - initial_weights[-1])

Number of units: 100
Epoch 1/400


2023-04-21 16:17:14.924735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 93/100 [==========================>...] - ETA: 0s - loss: 0.7084 - accuracy: 0.4946

InvalidArgumentError: Multiple Default OpKernel registrations match NodeDef '{{node ZerosLike}}': 'op: "ZerosLike" device_type: "DEFAULT" constraint { name: "T" allowed_values { list { type: DT_INT32 } } } host_memory_arg: "y"' and 'op: "ZerosLike" device_type: "DEFAULT" constraint { name: "T" allowed_values { list { type: DT_INT32 } } } host_memory_arg: "y"' [Op:ZerosLike]

In [29]:
%tensorboard --logdir logs/fit